In [1]:
# Module Importations
from scipy.stats import reciprocal
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import keras
import numpy as np
import time

print('keras.__version__=', keras.__version__)

Using TensorFlow backend.
keras.__version__= 2.3.1


In [2]:
# Project Module Importations
from src.data import load_data
from src.data import split_data
from src.models import keras_helpers
from src.models import model_evaluation

In [3]:
# Load dataset
original_dataset = load_data.load_motor_data()

In [4]:
# Split data into training / evaluation sets
training_set, evaluation_set = split_data.split_train_test(original_dataset, 0.8)

In [5]:
# Drop profile id data column
training_set = training_set.drop("profile_id", axis = 1)
evaluation_set = evaluation_set.drop("profile_id", axis = 1)

In [6]:
# Create rotor target
rotor_training_data = training_set.drop("pm", axis = 1)
rotor_label_data = training_set["pm"].copy()

rotor_test_data = evaluation_set.drop("pm", axis = 1)
rotor_test_label = evaluation_set["pm"].copy()

In [7]:
# Create train and test arrays
X_train, X_test, y_train, y_test = train_test_split(rotor_training_data, rotor_label_data, test_size = 0.2, random_state = 0)

In [9]:
# Multilayer Perceptron (Target - Rotor Temperature)

# Clear existing models
keras.backend.clear_session()

# Parameter distribution
param_distribs = { 
    "n_hidden": [1, 4, 8], 
    "n_neurons": np.arange(1, 100), 
    "learning_rate": reciprocal(3e-4, 3e-2)
    }

# Build model
wrapped_model = keras_helpers.wrap_model()

# Initialise Random Search
rnd_search_cv = RandomizedSearchCV(wrapped_model, param_distribs, n_iter = 10, cv = 3)

# Name model file
model_type = "MLP-RotorTemp"
model_id = keras_helpers.name_model(model_type)
filepath_full = keras_helpers.make_save_string(model_id)

# Set save & earlystop callbacks
earlystop_cb = keras.callbacks.EarlyStopping(patience = 5)
checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath = filepath_full, save_best_only = True)

# Train model
rnd_search_cv.fit( X_train, y_train, epochs = 100, validation_data =(X_test, y_test), callbacks =[checkpoint_cb, earlystop_cb])

Train on 106460 samples, validate on 39923 samples
Epoch 1/50
106460/106460 [==============================] - 8s 72us/step - loss: 0.5316 - val_loss: 0.2848
Epoch 2/50
 70560/106460 [==================>...........] - ETA: 2s - loss: 0.2581

KeyboardInterrupt: 

In [9]:
# Model Evaluation

# Load model
filepath = r'C:\Developer\electric_motor_thermal_modelling\Models\YC_model_2020_07_28-14_22_07.h5'

model = keras.models.load_model(filepath)

# Model Description

# Evaluation
start_time = time.process_time()
rotor_pred_eval = model.predict(rotor_test_data)
end_time = time.process_time()
print("Prediction Time:", (end_time-start_time))

model_name = "MLP - Rotor"
model_evaluation.evaluate_model(model_name, rotor_test_label, rotor_pred_eval)
#model.evaluate(rotor_test_data, rotor_test_label)

OSError: Unable to open file (unable to open file: name = 'C:\Developer\electric_motor_thermal_modelling\Models\YC_model_2020_07_28-14_22_07.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)